In [1]:
import numpy as np
import pandas as pd
import numpy as np
import scipy.sparse as sp
import random, datetime

import import_ipynb
from tqdm import tqdm as tqdm_pd
from tqdm import tqdm_notebook as tqdm
tqdm_pd.pandas()

import os, gc, logging
logger = logging.getLogger()

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import weightwatcher as ww

Using TensorFlow backend.


In [33]:
def norm_check(weight, N, M, receptive_field_size, 
                   lower = 0.5, upper = 1.5):
        
        kappa = np.sqrt( 2 / ((N + M)*receptive_field_size) )
        norm = np.linalg.norm(weight)  

        check1 = norm / np.sqrt(N*M)
        check2 = norm / (kappa*np.sqrt(N*M))
        
        print("CHECKS ",N,M,receptive_field_size, check1, check2, norm, np.sqrt(N*M))
        if (check2 > lower) & (check2 < upper):   
            print("RMT 2 normalization")
            return weight / (kappa * np.sqrt(N))  
        elif (check1 > lower) & (check1 < upper): 
            print("RMT 1normalization")
            return weight / np.sqrt(N)      
        else:
            return weight

### TODO: Tuesday

the problem with the Norm is that it is inconsistent...sometimes its on...sometimes not...it either has to always be on or always off...and switch based on the type of layer

1. mark kind of normalization in details
2. see if all layers are normalized or not
3. normalize Force or Test ?

CHECKS  40478 768 1 0.11040252530913634 15.85458681120852 615.5582 5575.581045953865


In [39]:
615/N

0.80078125

In [34]:
from sklearn.decomposition import TruncatedSVD
import powerlaw

def analyze(w):
    W = w.detach().numpy()
    M, N = np.min(W.shape), np.max(W.shape)
    svd = TruncatedSVD(n_components=M-1)
    svd.fit(W)
    sv = svd.singular_values_
    evals = (1/N)*sv**2
    fit = powerlaw.Fit(evals)
    fnorm = np.linalg.norm(W)
    print(np.max(sv*sv), np.max(evals))
    
    svd = TruncatedSVD(n_components=1)
    svd.fit(W)
    sv = svd.singular_values_
    evals = (1/N)*sv**2
    snorm = np.min(evals)
    
    
    return N, M, fit.alpha,fnorm, snorm

In [40]:
for im, m in enumerate(gpt_model.modules()):
    if im>0  :
        try:
            w = m.weight
            print(im,m)


            W = w.detach().numpy()
        
            M, N = np.min(W.shape), np.max(W.shape)
            wn= norm_check(W,N,M,1)
            #print("norm comparison", np.linalg.norm(W),np.linalg.norm(wn))
            #N, M, alpha,  fnorm, snorm =  analyze(w)
            #print(im,  N, M, alpha, fnorm, snorm)
        except:
            pass

1 Embedding(40478, 768)
CHECKS  40478 768 1 0.11040252530913634 15.85458681120852 615.5582 5575.581045953865
2 Embedding(512, 768)
CHECKS  768 512 1 0.04564571159323274 1.1547553124301495 28.623028 627.0693741524935
RMT 2 normalization
7 Conv1D()
CHECKS  2304 768 1 0.12320485338300652 4.828624393964477 163.88895 1330.2150202128978
8 Conv1D()
CHECKS  768 768 1 0.034841341276963554 0.9655515727287614 26.75815 768.0
RMT 2 normalization
11 BertLayerNorm()
CHECKS  768 768 1 0.044536083936691284 1.2342201623760232 34.203712 768.0
RMT 2 normalization
13 Conv1D()
CHECKS  3072 768 1 0.05475567777951559 2.399273589706006 84.10472 1536.0
14 Conv1D()
CHECKS  3072 768 1 0.05248806377251943 2.299911722238288 80.621666 1536.0
16 BertLayerNorm()
CHECKS  768 768 1 0.01620267704129219 0.44902175766636554 12.443656 768.0
19 Conv1D()
CHECKS  2304 768 1 0.06703046567366674 2.6270470099460255 89.16493 1330.2150202128978
20 Conv1D()
CHECKS  768 768 1 0.03901206453641256 1.0811340461635606 29.961266 768.0
RMT

In [20]:
details[details.level==ww.LEVEL.SLICE].head(2)

,layer_type,N,M,level,comment,norm,lognorm,alpha,alpha_weighted,spectralnorm,softrank,softranklog,softranklogratio,logpnorm,NxM
layer_id,,,,,,,,,,,,,,,
1,CONV1D,40478,768,LEVEL.SLICE,Slice level,615.558,2.78927,2.99837,15.0206,102234,3.70631,0.568941,1.11357,15.0209,31087104
2,CONV1D,768,512,LEVEL.SLICE,Slice level,26.1291,1.41712,1.4903,3.27,156.381,4.36582,0.640066,1.29171,3.76026,393216


In [ ]:
results = watcher.analyze(layers=[1,2],alphas=True, softranks=True, spectralnorms=True)

summary =  watcher.get_summary()

details  = watcher.get_details(results=results)

In [ ]:
details